# Generation and debugging script for the Multi Channel Speech Enhancement Dataset

In [ ]:
import numpy as np
import pyroomacoustics as pra
from scipy.io import wavfile
import IPython
import os
import matplotlib.pyplot as plt
import sys
sys.path.append('..')
%reload_ext autoreload
%autoreload 2

## Split the source datas for train and val

In [ ]:
# load all clean speeches
clean_root = '../data/datasets/datasets_fullband/clean_fullband/read_speech'
clean_records = os.listdir(clean_root)
clean_records.sort()
print(len(clean_records))

# split train and val clean speeches set
data_split = {'train':20,'val':1}
n_clean_train = int(len(clean_records)*data_split['train']/sum(data_split.values()))

rs = np.random.RandomState(123)

rs.shuffle(clean_records)
cleans_train = clean_records[:n_clean_train]
cleans_val = clean_records[n_clean_train:]
print(f'clean train:val = {len(cleans_train)}:{len(cleans_val)}')

# load all noises
noise_root = '../data/datasets/datasets_fullband/noise_fullband'
noise_records = os.listdir(noise_root)
noise_records.sort()
print(len(noise_records))

# split train and val noise speeches set
n_noise_train = int(len(noise_records)*data_split['train']/sum(data_split.values()))

rs.shuffle(noise_records)
noises_train = noise_records[:n_noise_train]
noises_val = noise_records[n_noise_train:]
print(f'noise train:val = {len(noises_train)}:{len(noises_val)}')

write_to_files = True
if write_to_files:
    # write to files
    with open('../data/datasets/datasets_fullband/cleans_train', 'w') as f:
        for i,x in enumerate(cleans_train):
            if i>0:
                f.write('\n')
            f.write(x)
    with open('../data/datasets/datasets_fullband/noises_train', 'w') as f:
        for i,x in enumerate(noises_train):
            if i>0:
                f.write('\n')
            f.write(x)
    with open('../data/datasets/datasets_fullband/cleans_val', 'w') as f:
        for i,x in enumerate(cleans_val):
            if i>0:
                f.write('\n')
            f.write(x)
    with open('../data/datasets/datasets_fullband/noises_val', 'w') as f:
        for i,x in enumerate(noises_val):
            if i>0:
                f.write('\n')
            f.write(x)

In [ ]:
fs, clean = wavfile.read(os.path.join(clean_root,cleans_val[0]))
print(fs,clean.dtype)
IPython.display.display(IPython.display.Audio(clean, rate=fs))
fs, noise = wavfile.read(os.path.join(noise_root,noises_val[0]))
IPython.display.display(IPython.display.Audio(noise , rate=fs))

## Sample generation play ground



In [ ]:
import json
from dataset.mcse_dataset import generate_random_noisy_for_speech
with open('mcse_dataset_settings_v2.json','r') as f:
    opt = json.load(f)
fs = opt['audio']['fs']
sample = generate_random_noisy_for_speech(opt, 8, cleans_val[0], noises_val, clean_root, noise_root)
meta = sample['meta']
room = sample['room']
freefield = sample['freefield']
noisy = sample['noisy']
clean = sample['clean']
fig, ax = room.plot(img_order=0)
print(ax)
ax.view_init(elev=90,azim=-90)
ax.set_xlabel('x')
ax.set_ylabel('y')
print(meta)

In [ ]:
def show_audio(audio,fs,name):
    print(f'{name} shape={audio.shape}')
    plt.plot(audio)
    plt.show()
    IPython.display.display(IPython.display.Audio(audio, rate=fs))
    
# plt.plot(room.rir[0][0])
# plt.show()
# plt.plot(freefield.rir[0][0])
# plt.show()
print(noisy.shape)
print(clean.shape)
show_audio(noisy[0],fs,'noisy[0]')
show_audio(clean,fs,'clean')

In [ ]:

show_audio(clean,fs,'clean')
show_audio(noisy[0],fs,'noisy[0]')
show_audio(noisy[0]-clean,fs,'noisy[0]-clean')


## Generate val samples

We only generate validation samples while training samples are generated online.

In [ ]:
import pickle
from tqdm import tqdm

with open('mcse_dataset_settings_v2.json','r') as f:
    opt = json.load(f)
data_val_root = '../data/datasets/mcse_val_setting2'
if not os.path.exists(data_val_root):
    os.makedirs(data_val_root)
    
noisy_val_root = os.path.join(data_val_root,'noisy')
if not os.path.exists(noisy_val_root):
    os.makedirs(noisy_val_root)
    
clean_val_root = os.path.join(data_val_root,'clean')
if not os.path.exists(clean_val_root):
    os.makedirs(clean_val_root)
    
meta_val_root = os.path.join(data_val_root,'meta')
if not os.path.exists(meta_val_root):
    os.makedirs(meta_val_root)

def write_meta(meta_root,name):
    with open(os.path.join(meta_root,f'{name}.pickle'),'wb') as f:
        pickle.dump(meta,f)

def load_meta(meta_root,name):
    with open(os.path.join(meta_root,f'{name}.pickle'),'rb') as f:
        loaded_meta = pickle.load(f)
    return loaded_meta

def write_audio(audio, fs, path):
    audio = np.clip(audio,-1,1)*np.iinfo(np.int16).max-1
    wavfile.write(path, fs, audio.astype(np.int16))

clip_seconds = 8
print(fs)
for i in tqdm(range(len(cleans_val))):
    clean = cleans_val[i]
    sample = generate_random_noisy_for_speech(opt, clip_seconds, clean, noises_val, clean_root, noise_root)
    meta = sample['meta']
    room = sample['room']
    freefield = sample['freefield']
    noisy = sample['noisy']
    clean = sample['clean']
    name = f'{i:04}'
    write_meta(meta_val_root,name)
    write_audio(noisy.T,fs,os.path.join(noisy_val_root,f'{name}.wav'))
    write_audio(clean,fs,os.path.join(clean_val_root,f'{name}.wav'))
    

    